In [1]:
import os
import sys
src_path = os.path.abspath(os.path.join(os.getcwd(), "../..", "backend"))
if src_path not in sys.path:
    sys.path.append(src_path)

In [2]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
import os
from azure.identity.aio import (ClientSecretCredential, DefaultAzureCredential,
                                get_bearer_token_provider)
from autogen_agentchat.agents import UserProxyAgent
from helpers.fmputils import *
from helpers.yfutils import *
from autogen_core.tools import FunctionTool
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from helpers.coding import *
from helpers.text import *
from datetime import date, timedelta, datetime
from helpers.reports import *
from helpers.charting import *
from textwrap import dedent
from helpers.analyzer import *
import tempfile
from helpers.summarizeutils import summarize, summarizeTopic
from helpers.options import *

In [3]:
symbol = "SPY"

In [4]:
today = pd.to_datetime(datetime.now())
gexladderDf, gexFlowLevelsDf, df = get_levels(symbol)

Getting GEX and Flow Levels
Getting Gex and Flow Levels
Getting CBOE Option Data for SPY
Underlying index price: 603.45
Calculating GEX Ladder
Calculating Flow Levels
Calculating Gamma Levels
Calculating Vanna Levels
Calculating Additional Gex Levels
Calculating All Expiration Levels
Calculating First Expiration Levels
Calculating Second Expiration Levels
Calculating Weekly Expiration Levels
Calculating Monthly Expiration Levels


In [5]:
gexFlowLevelsDf

,symbol,expiration,processTime,vol_call,vol_put,oi_call,oi_put,vol_avg,oi_avg,vol_resistance1,...,tot_oi_m,zero_gamma1_m,gamma_flip1_m,gamma_flip2_m,pain_volume_strike_m,pain_oi_strike_m,zero_pos_strike_m,zero_neg_strike_m,tot_vol_ratio_m,tot_oi_ratio_m
0,SPY,All,2025-01-21 16:16:00,607.36,587.64,606.68,573.03,597.5,589.86,602.43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SPY,First,2025-01-21 16:16:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SPY,Second,2025-01-21 16:16:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SPY,Weekly,2025-01-21 16:16:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SPY,Monthly,2025-01-21 16:16:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.6563,556.0,610.0,597.0,594.0,575.0,630.0,555.0,0.4789,0.2259


In [6]:
df

,ExpirationDate,Calls,CallLastSale,CallNet,CallBid,CallAsk,CallVol,CallIV,CallDelta,CallGamma,...,CallWall,PutWall,CumCallBidOI,CumPutBidOI,CumCallBidVol,CumPutBidVol,CumTotalBidOI,CumTotalBidVol,daysTillExp,IsThirdFriday
0,2025-01-22 16:00:00,SPY250122C00400000,0.00,0.00,202.47,203.69,0.0,2.2294,0.9999,0.0000,...,0.0000,160.0,-1.198198e+08,-30945155.3,-5118515.83,-4838252.53,-1.507650e+08,-9956768.36,0.003817,False
1,2025-01-22 16:00:00,SPY250122C00405000,0.00,0.00,197.49,198.69,0.0,2.1308,0.9999,0.0000,...,0.0000,0.0,-1.198198e+08,-30945155.3,-5118515.83,-4838252.53,-1.507650e+08,-9956768.36,0.003817,False
2,2025-01-22 16:00:00,SPY250122C00410000,0.00,0.00,192.47,193.69,0.0,2.0257,0.9999,0.0000,...,0.0000,0.0,-1.198198e+08,-30945155.3,-5118515.83,-4838252.53,-1.507650e+08,-9956768.36,0.003817,False
3,2025-01-22 16:00:00,SPY250122C00415000,0.00,0.00,187.49,188.69,0.0,1.9064,0.9999,0.0000,...,0.0000,1.0,-1.198198e+08,-30945155.3,-5118515.83,-4838252.53,-1.507650e+08,-9956768.36,0.003817,False
4,2025-01-22 16:00:00,SPY250122C00420000,0.00,0.00,182.49,183.69,0.0,1.7407,0.9999,0.0000,...,0.0000,0.0,-1.198198e+08,-30945155.3,-5118515.83,-4838252.53,-1.507650e+08,-9956768.36,0.003817,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4454,2027-12-17 16:00:00,SPY271217C00880000,0.00,0.00,5.88,8.05,0.0,0.1247,0.1260,0.0016,...,0.0016,0.0,1.198065e+08,30945155.3,5117431.03,4838252.53,1.507517e+08,9955683.56,2.893130,True
4455,2027-12-17 16:00:00,SPY271217C00885000,0.00,0.00,5.47,7.68,0.0,0.1246,0.1208,0.0016,...,0.0016,0.0,1.198065e+08,30945155.3,5117431.03,4838252.53,1.507517e+08,9955683.56,2.893130,True
4456,2027-12-17 16:00:00,SPY271217C00890000,4.47,0.00,5.13,7.26,0.0,0.1244,0.1160,0.0016,...,3.0016,0.0,1.198065e+08,30945155.3,5117431.03,4838252.53,1.507517e+08,9955683.56,2.893130,True
4457,2027-12-17 16:00:00,SPY271217C00895000,3.45,0.00,4.79,6.91,0.0,0.1243,0.1115,0.0015,...,1.0015,0.0,1.198066e+08,30945155.3,5117431.03,4838252.53,1.507517e+08,9955683.56,2.893130,True
